In [1]:
from notebook_env import * 

In [17]:
epsilons_lin = np.linspace(1e-5, 1e-1, num = 20)
epsilons_log = np.logspace(-5,-1, num = 20)

In [20]:
k_inner, k_outer = 5,5
n_reps = 100
patience = n_reps*(1+k_inner)*k_outer/100
quantiles = [0.7,0.8,0.9,0.95]

In [ ]:
# Define the experiment
def run_single_early_stopping_experiment(epsilon, quantile):
    exp = nested_cv_intervals_with_early_stopping.CvIntervalsWithEarlyStoppingTest()